In [1]:
import json
from datetime import datetime
from os import getenv

from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, Field

load_dotenv()

# Initialize the language model
llm = OpenAI(
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url=getenv("OPENROUTER_BASE_URL"),
)

## Unstructured Output

In [2]:
# Define your prompt
prompt = "Can you tell me about Harry Potter's profile?"

# Make the LLM call
response = llm.chat.completions.create(
    model="google/gemini-pro",
    messages=[{"role": "user", "content": prompt}],
)

# Print the result
print(response.choices[0].message.content)

## Harry Potter Profile: 

**Full name:** Harry James Potter
**Birthday:** July 31, 1980
**Blood status:** Half-blood
**Parents:** Lily Evans and James Potter (deceased)
**Raised by:** Petunia Dursley and Vernon Dursley (aunt and uncle)
**Marital Status:** Married to Ginny Weasley
**Children:** James Sirius, Albus Severus, and Lily Luna
**House:** Gryffindor
**Occupation:** Auror (formerly) Head of the Auror Office (formerly)

### Appearance: 
* Messy black hair
* Bright green eyes
* Lightning bolt-shaped scar on his forehead (from Lord Voldemort) 
* Wears round glasses

### Personality:
* Brave
* Loyal
* Selfless
* Kind-hearted
* Determined

### Abilities: 
* Skilled wizard 
* Excellent Defense Against the Dark Arts skills
* Speaks Parseltongue (the ability to speak to snakes)
* Can fly without a broom (due to the scar on his head)
* Famous as "The Boy Who Lived" 

### Important Events in Harry's Life:
* **1981:** Survives Lord Voldemort's attack and becomes "The Boy Who Lived" 
* **1

## Structured Output

In [3]:
class Person(BaseModel):
    name: str = Field(description="The person's full name")
    date_of_birth: datetime = Field(
        description="The person's date of birth in ISO 8601 format (YYYY-MM-DD)"
    )
    occupation: str = Field(description="The person's current job or profession")


def get_person_info() -> Person:
    """Get information about a person from LLM response."""

    return Person(name="", age=0, occupation="")


# Create the function description for the API
function_description = {
    "name": "get_person_info",
    "description": "Get information about a person from LLM response.",
    "parameters": Person.model_json_schema(),
}

# Make the LLM call
response = llm.chat.completions.create(
    model="google/gemini-pro",
    messages=[{"role": "user", "content": prompt}],
    functions=[function_description],
    function_call={"name": "get_person_info"},
)

# Extract the function call arguments
function_call = response.choices[0].message.tool_calls[0].function
arguments = json.loads(function_call.arguments)

# Create a Person object from the arguments
person = Person(**arguments)

# Print the result
print(f"Name: {person.name}")
print(f"DoB: {person.date_of_birth}")
print(f"Occupation: {person.occupation}")

Name: Harry Potter
DoB: 1980-07-31 00:00:00
Occupation: Wizard
